# description

**purpose**: search multiple dates and itineraries from skiplagged 

# setup

In [ ]:
import requests
from time import sleep
import datetime
from datetime import date, timedelta
import geopy.distance
import airportsdata
import json



# manual

In [ ]:
# list of closest source airports
airports = airportsdata.load('IATA')  # key is IATA code

airport_code = 'SFO'
distance_km = 100
relevant_airports = get_closest_airports(airport_code, distance_km)
relevant_airports

In [ ]:
# distance between 2 airports
a = 'MUC'
src = (airports[a]['lat'], airports[a]['lon'])
a = 'SOF'
dst = (airports[a]['lat'], airports[a]['lon'])
dist = geopy.distance.distance(src, dst).km
dist


In [ ]:
# list of closest destination airports

dst = 'MUC'
distance_km = 1200.0
dst_airports = get_closest_airports(dst, distance_km)
len(dst_airports), 'SOF' in dst_airports

# functions

In [ ]:
def skiplagged_api_get(url, seconds=15, wait_seconds=5):
    i = 0
    call_times = seconds/wait_seconds
    while i < call_times:
        response = requests.get(url)
        response_dict = response.json()
        sleep(wait_seconds)
        i+=1
    return response_dict

In [ ]:
def process_response(response_dict, n=5):
    details = []
    flights = []
    cheapest_n_flights = response_dict['itineraries']['outbound'][:n]
    for flight in cheapest_n_flights:
        flight_id = flight['flight']
        details = response_dict['flights'][flight_id]
        departure = response_dict['flights'][flight_id]['segments'][0]['departure']['time']
        arrival = response_dict['flights'][flight_id]['segments'][-1]['arrival']['time']
        hours = response_dict['flights'][flight_id]['duration']/60/60
        usd_price = flight['one_way_price']/100
        flight_dict = {'flight_id': flight_id, 
                       'cost': usd_price, 
                       'duration': hours, 
                       'departure': departure, 
                       'arrival': arrival}
        details = {flight_id: details}
        flights.append(flight_dict)
    return flights, details

In [ ]:
def get_airports_by_country(country):
    airports = airportsdata.load('IATA')  # key is IATA code
    airports_by_country = [a for a in airports if airports[a]['country'] == country]
    return airports_by_country

In [ ]:
def get_closest_airports(airport_code, distance_km):
    airports = airportsdata.load('IATA')  # key is IATA code
    src = (airports[airport_code]['lat'], airports[airport_code]['lon'])
    relevant_airports = []
    for a in airports:
        dst = (airports[a]['lat'], airports[a]['lon'])
        dist = geopy.distance.distance(src, dst).km
        if dist <= distance_km:
            relevant_airports.append(a)

    return relevant_airports

# execute

In [ ]:
# save location

SAVE_PATH = '/home/alex/Downloads/itineraries/'


In [ ]:
# SPACE: final list of source and destination airports

src_airports = ['OAK', 'SFO', 'SJC']
dst_airports = ['SOF', 'BEG', 'SKP', 'IST', 'OTP']

In [ ]:
# TIME: list of dates

start = '2022-05-26'
end = '2022-06-23'
#end = '2022-05-26'

fmt = '%Y-%m-%d'

start_date = datetime.datetime.strptime(start, fmt)
end_date = datetime.datetime.strptime(end, fmt)

delta = end_date - start_date   # returns timedelta

dpt_dates = [(start_date + timedelta(days=i)).strftime(fmt) for i in range(delta.days + 1)]
    
len(dpt_dates)

In [ ]:
def get_all_combinations(src_airports, dst_airports, dpt_dates, save_path):
    results = []
    N = len(src_airports)*len(dst_airports)*len(dpt_dates)
    i = 1
    dt = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    for src in src_airports:
        for dst in dst_airports:
            for dpt in dpt_dates:
                print(f'processing {i}/{N}: {src}, {dst}, {dpt}')
                rtn = ''
                fmt = 'v3'
                alt = '1'
                cld = '0'
                url = f"https://skiplagged.com/api/search.php?from={src}&to={dst}&depart={dpt}&return={rtn}&format={fmt}&counts[adults]={alt}&counts[children]={cld}"
                response_dict = skiplagged_api_get(url, seconds=15, wait_seconds=5)
                best_flights, _ = process_response(response_dict, n=5)
                
                for bf in best_flights:
                    result = {}
                    result.update({'src': src, 'dst': dst, 'depart_date': dpt})
                    result.update(bf)
                    results.append(result)
                    with open(f"{save_path}itinerary_{dt}.txt", "a") as f:
                        f.write(json.dumps(result))
                
                i+=1
                
    return results

In [ ]:
results = get_all_combinations(src_airports, dst_airports, dpt_dates, save_path=SAVE_PATH)
sorted_results = sorted(results, key = lambda i: i['cost'])
# 435 combos take 2h 11m 47 s

In [ ]:
sorted_results